In [56]:
from hushh.catalog import Catalog
from hushh.hcf import VibeMode
import numpy as np

In [45]:
cat = Catalog.read_hcf("catalog.hcf")

In [46]:
pv = cat.ProductVibes()

In [68]:
pv = cat.ProductVibes()
all_embeddings = []
for idx in range(0,pv.FlatBatchesLength()):
    batch = pv.FlatBatches(idx)
    type = batch.Type()
    embeddings = batch.FlatTensorAsNumpy()
    print(embeddings.shape)
    embeddings = embeddings.reshape(batch.ShapeAsNumpy())
    all_embeddings.append(embeddings)
all_embeddings = np.concatenate(all_embeddings)
all_embeddings.shape

(5120000,)
(5120000,)
(5120000,)
(5120000,)
(5120000,)
(5120000,)
(5120000,)
(5120000,)
(2270208,)
(2270208,)


(88868, 512)

In [67]:
dir(batch)

['FlatTensor',
 'FlatTensorAsNumpy',
 'FlatTensorIsNone',
 'FlatTensorLength',
 'GetRootAs',
 'GetRootAsFlatEmbeddingBatch',
 'Id',
 'Init',
 'Sequence',
 'Shape',
 'ShapeAsNumpy',
 'ShapeIsNone',
 'ShapeLength',
 'Type',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_tab']